In [1]:
import pyarrow.dataset
import pyarrow.fs
from azure.identity import ManagedIdentityCredential
from azure.core.credentials import AccessToken

In [ ]:
%pip install pyarrowfs_adlgen2

In [ ]:
import pyarrowfs_adlgen2

In [ ]:
# Add pyarrow dependency at session or spark pool level but do not install after creating a session.
# Check pyarrow version 12.0.0
import pkg_resources
for d in pkg_resources.working_set:
     print(d)

In [ ]:
from datetime import datetime

dt = datetime.today()  
seconds = dt.timestamp()

#azure-identity does not work for the jobs that run in synapse workspace.
# For more information, you can check https://learn.microsoft.com/en-us/azure/synapse-analytics/synapse-service-identity?context=%2Fazure%2Fsynapse-analytics%2Fcontext%2Fcontext.

# Alternative is to create a token from mssparkutils
# Assign the spoof_token to ManagedIdentityCredential class object
class spoof_token:
    def get_token(*args, **kwargs):
        return AccessToken(
            token=mssparkutils.credentials.getToken(audience="storage"),
            # some random time in future... synapse doesn't provide way to get a expires_on value for a token
            expires_on=seconds+(60*100) 
        )

credential = ManagedIdentityCredential()
credential._credential = spoof_token()

In [ ]:
# Replace these variables with your actual account_name and account_key
ACCOUNT_NAME = "syncdldev01"
ACCOUNT_KEY = ""

# Replace this with the container name and Parquet file path in ADLS Gen2
CONTAINER_NAME = "tlfs"
PARQUET_FILE_PATH = 'poc/parquet/nyc_yellow_taxi_trips/puYear=2001/puMonth=1/part-00493-127a558e-137b-4f6b-8c6e-904dc74a264e.c000.snappy.parquet'

In [ ]:
#Use the credential from spoof_token class
handler = pyarrowfs_adlgen2.FilesystemHandler.from_account_name(
   ACCOUNT_NAME, CONTAINER_NAME, credential)
   
fs = pyarrow.fs.PyFileSystem(handler)

import pyarrow as pa
import pyarrow.parquet as pq

metadata = pq.read_metadata(PARQUET_FILE_PATH,filesystem=fs)

In [ ]:
print(metadata)
rg = metadata.row_group(0)